Задание 1
Импортируйте библиотеки pandas и numpy.

In [4]:
import numpy as np
import pandas as pd

Загрузите "Boston House Prices dataset" из встроенных наборов данных библиотеки sklearn. Создайте датафреймы X и y из этих данных.

In [5]:
from sklearn.datasets import load_boston
boston = load_boston()

In [6]:
boston.keys()

dict_keys(['data', 'target', 'feature_names', 'DESCR', 'filename'])

In [7]:
X = pd.DataFrame(boston.data, columns=boston.feature_names)
y = pd.DataFrame(boston.target, columns=['Price'])

Разбейте эти датафреймы на тренировочные (X_train, y_train) и тестовые (X_test, y_test) с помощью функции train_test_split так, чтобы размер тестовой выборки составлял 30% от всех данных, при этом аргумент random_state должен быть равен 42.

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

Создайте модель линейной регрессии под названием lr с помощью класса LinearRegression из модуля sklearn.linear_model.

In [9]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

Обучите модель на тренировочных данных (используйте все признаки) и сделайте предсказание на тестовых.

In [10]:
lr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [11]:
y_predict = lr.predict(X_test)

In [12]:
check_table = pd.DataFrame({'y_test': y_test['Price'], 'y_predict': y_predict.flatten()})
check_table.head()

,y_test,y_predict
173,23.6,28.648960
274,32.4,36.495014
491,13.6,15.411193
72,22.8,25.403213
452,16.1,18.855280


Вычислите R2 полученных предказаний с помощью r2_score из модуля sklearn.metrics

In [13]:
from sklearn.metrics import r2_score
r2_score(y_test, y_predict)

0.7112260057484912

Задание 2
Создайте модель под названием model с помощью RandomForestRegressor из модуля sklearn.ensemble.
Сделайте агрумент n_estimators равным 1000, max_depth должен быть равен 12 и random_state сделайте равным 42.

In [14]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=1000, max_depth=12, random_state=42)

Обучите модель на тренировочных данных аналогично тому, как вы обучали модель LinearRegression, 
но при этом в метод fit вместо датафрейма y_train поставьте y_train.values[:, 0],
чтобы получить из датафрейма одномерный массив Numpy, так как для класса RandomForestRegressor в данном методе для аргумента y предпочтительно применение массивов вместо датафрейма. Сделайте предсказание на тестовых данных и посчитайте R2. Сравните с результатом из предыдущего задания.

In [15]:
model.fit(X_train, y_train.values[:, 0])

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=12,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=None,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

In [16]:
y_RFR_predict = model.predict(X_test)

In [17]:
check_RFR_table = pd.DataFrame({'y_test': y_test['Price'], 'y_RFR_predict': y_RFR_predict.flatten()})
check_RFR_table.head()

,y_test,y_RFR_predict
173,23.6,22.846138
274,32.4,31.156114
491,13.6,16.297226
72,22.8,23.821036
452,16.1,17.212148


In [18]:
r2_score(y_test, y_RFR_predict)

0.8749965273218174

Напишите в комментариях к коду, какая модель в данном случае работает лучше.
Коэффициент R2 на модели Случайный лес составил 0.875, а на линейной регрессии 0.711 => модель Random forest эффективнее

* Задание 3
Вызовите документацию для класса RandomForestRegressor,
найдите информацию об атрибуте feature_importances_.

С помощью этого атрибута найдите сумму всех показателей важности,
установите, какие два признака показывают наибольшую важность.

In [19]:
?RandomForestRegressor()

In [23]:
feature_importance = model.feature_importances_
feature_importance

array([0.03211748, 0.00154999, 0.0070941 , 0.0011488 , 0.01436832,
       0.40270459, 0.01424477, 0.06403265, 0.00496762, 0.01169177,
       0.01808961, 0.0123114 , 0.41567892])

In [38]:
#два последних (= максимальных по важности) значения в сортированном массиве из показателей значимости признаков
feature_importance[feature_importance.argsort()[-1:-3:-1]]

array([0.41567892, 0.40270459])

In [41]:
#наименование признаков с максимальной важностью
X_train.columns[feature_importance.argsort()[-1:-3:-1]]

Index(['LSTAT', 'RM'], dtype='object')

In [44]:
#найдите сумму всех показателей важности
feature_importance.sum()

0.9999999999999994

* Задание 4 датасет Credit Card Fraud Detection
Для этого датасета мы будем решать задачу классификации - будем определять, какие из транзакциции по кредитной карте являются мошенническими. Мы будем вычислять AUC, то есть площадь под кривой ROC. Импортируйте из соответствующих модулей RandomForestClassifier, GridSearchCV и train_test_split.